# Simplex method

We are going to illustrate simplex tableau method for linear programing using the following example - WG:
$$\max z = 3 x_1 + 5 x_2$$
s.t.
$$\begin{array}{lll}
x_1 & & \le 4 \\
& 2 x_2 &\le 12 \\
3x_1 & + 2x_2 & \le 18\\
x_1, & x_2 & \ge 0
\end{array}
$$

In [1]:
# import package
import numpy as np
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
import pandas as pd
import warnings
#ignore by message
warnings.filterwarnings("ignore", message="divide by zero encountered in true_divide")

## Pivoting

Pivoting with $(i,j)$-entry means converting the matrix by multiple EROS so that the resulting matrix has all zeros in $j$-column, but one in $(i,j)$-entry. This is the key step in updating basic feasible solution.

In [2]:
# pivoting with (i,j)
def pivoting(A, i, j):
    A[i] = A[i] / A[i, j]  # scale to get one in (i,j)
    n_rows, _ = A.shape
    for k in range(n_rows):
        if k == i:
            continue  # skip i-row
        A[k] = A[k] - A[i] * A[k, j]  # replacement to get zero

# Initial tableau

In [3]:
# input initial tableau
M_mat = np.array([
    [1, -3, -5, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 4],
    [0, 0, 2, 0, 1, 0, 12],
    [0, 3, 2, 0, 0, 1, 18]
], dtype=float)

print(f'=======================')
print(f'initial tableau is')
print(pd.DataFrame(M_mat))

initial tableau is
     0    1    2    3    4    5     6
0  1.0 -3.0 -5.0  0.0  0.0  0.0   0.0
1  0.0  1.0  0.0  1.0  0.0  0.0   4.0
2  0.0  0.0  2.0  0.0  1.0  0.0  12.0
3  0.0  3.0  2.0  0.0  0.0  1.0  18.0


In [4]:
row_n, col_n = M_mat.shape - np.array([1,2])
print(f'number of constraints is {row_n}')
print(f'number of decision variables is {col_n}')

number of constraints is 3
number of decision variables is 5


In [5]:
# Do optimal test
# If not pass the tes, the pivot column is the one corresponding to the most negative coefficient in zero row
optimal_test = min(M_mat[0, range(1, col_n + 1)])

if optimal_test < 0:
    pivot_c = np.argmin(M_mat[0, range(1, col_n + 1)]) + 1
    print(f'new pivot column is {pivot_c}')
else:
    print(f'=================================')
    print(f'pass the optimal test')
    print(f'optimal value is {M_mat[0, -1]}')
    print(f'The final tableau is \n {pd.DataFrame(M_mat)}')
    print(f'=================================')

new pivot column is 2


In [6]:
# pivot row is corresponding to the minimum positive ratio
# minimum ratio test
def min_ratio_test(ratio_list):
    pivot_row = 0
    min_ratio = np.inf
    for i in range(len(ratio_list)):
        if 0 < ratio_list[i] < min_ratio:
            pivot_row = i + 1
            min_ratio = ratio_list[i]
    if min_ratio is np.inf:
        return 0  # no leaving variable
    else:
        return pivot_row

ratios = np.divide(M_mat[range(1, row_n + 1), -1], M_mat[range(1, row_n + 1), pivot_c])

pivot_r = min_ratio_test(ratios)
if pivot_r is 0:
    print(f'no leaving variable, here is the ratio list {ratios}')
else:
    print(f'pivot row is {pivot_r}')

pivot row is 2


In [7]:
# Pivoting
pivoting(M_mat, pivot_r, pivot_c)
print(f'new tableau is:')
print(f'=======================')
print(pd.DataFrame(M_mat))

new tableau is:
     0    1    2    3    4    5     6
0  1.0 -3.0  0.0  0.0  2.5  0.0  30.0
1  0.0  1.0  0.0  1.0  0.0  0.0   4.0
2  0.0  0.0  1.0  0.0  0.5  0.0   6.0
3  0.0  3.0  0.0  0.0 -1.0  1.0   6.0


Ex. Repeat above algorithm until pass the optimal test